<a href="https://colab.research.google.com/github/KateGrim/Sheets/blob/main/EmailTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from oauth2client import client
from google.colab import files
import time
import os
import smtplib
from email.message import EmailMessage
import json
from openpyxl import Workbook, load_workbook

In [ ]:
login = files.upload()

Saving smtpcred.json to smtpcred.json


In [ ]:
uploaded = files.upload()

Saving client_secret.json.API to client_secret.json.API


In [ ]:
# define scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add the credentials from the json key file
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json.API', scope)

my_client = gspread.authorize(creds)

In [ ]:
def open_and_save_tabs(url_string):
  amdf_list = []
  email_list = []
  col_list = [
              'CSM',
              'AM',
              'Date Found',
              'Hot Tour Move',
              'Prop ID',
              'Lead ID',
              'Community',
              'Org',
              'Resident',
              'Contact',
              'Advisor',
              'RM',
              'HCAE',
              'Notes',
              'Blocker',
              'Next Steps',
              'Email'
  ]

  tab_nums = [0,1]

  open_wkbk = my_client.open_by_url(url_string)
  
  for tab in tab_nums:
    am = open_wkbk.get_worksheet(tab)
    hots = am.get('A:Q')      #Q is email col I manually added to populate email address the file needs to be sent to
    am_df = pd.DataFrame(hots, columns = col_list)
    email_address = am_df.iloc[1,16]
    am_df = am_df.drop(columns='Email')
    am_df = am_df[(am_df['CSM'] != 'CSM') & (am_df['CSM'] != '')]
    amdf_list.append((am_df,email_address))
     
  return amdf_list


In [ ]:
def format_file(file):
    wb = load_workbook(file)
    ws = wb.worksheets[0]
    ws.column_dimensions['A'].width = 20
    ws.column_dimensions['B'].width = 20
    ws.column_dimensions['C'].width = 20
    ws.column_dimensions['D'].width = 15
    ws.column_dimensions['E'].width = 10
    ws.column_dimensions['F'].width = 10
    ws.column_dimensions['G'].width = 40
    ws.column_dimensions['H'].width = 40
    ws.column_dimensions['I'].width = 22
    ws.column_dimensions['J'].width = 22
    ws.column_dimensions['K'].width = 22
    ws.column_dimensions['L'].width = 22
    ws.column_dimensions['M'].width = 22
    ws.column_dimensions['N'].width = 55
    ws.column_dimensions['O'].width = 50
    ws.column_dimensions['P'].width = 50
    wb.save(filename = attachment[0])

In [ ]:
all_attachments = []
login_creds = json.load(open('smtpcred.json'))
my_smtp = smtplib.SMTP('smtp-mail.outlook.com', port='587')
my_smtp.ehlo()
my_smtp.starttls()
my_smtp.login(login_creds['user'],login_creds['password'])


for df in open_and_save_tabs('https://docs.google.com/spreadsheets/d/14kntdH4Izm47kBCGV6lfyCIf_2hkpDAy2yjI-MBR1mI/edit?usp=sharing'):
  sheet_name = df[0].iloc[1,1] + '.xlsx'
  path_name = './' + sheet_name
  df[0].to_excel(path_name, index=False)
  all_attachments.append((sheet_name, df[1]))
  
for attachment in all_attachments:   
  format_file(attachment[0])
  msg = EmailMessage()
  msg['Subject'] = 'Monday Update'
  msg['To'] = attachment[1]
  msg['From'] = 'heather.knudson@aplaceformom.com'
  msg.set_content("""\
  Hello!

  Here is your curated list of CSM Hot Leads for this week.

  --Heather
  """)


  with open(attachment[0], 'rb') as f:
    to_send = f.read()
    msg.add_related(to_send, maintype='application', subtype='xlsx', filename=attachment[0])
  
    my_smtp.send_message(msg)


my_smtp.quit()

(221, b'2.0.0 Service closing transmission channel')

In [ ]:
  # this is what the above syntax actually is, just for reference; allows us to name the sheet exactly what we want:
    # df.to_excel('./Adriana Lamas.xlsx', index=False)
    # files.download('Adriana Lamas.xlsx')